In [105]:
import pandas as pd
import numpy as np

## Loading initial datasets and filtering

In [106]:
results = pd.read_csv('data/ecotox/results.txt',sep='\|', engine='python')
results.head()

,result_id,test_id,sample_size_mean_op,sample_size_mean,sample_size_min_op,sample_size_min,sample_size_max_op,sample_size_max,sample_size_unit,sample_size_comments,...,dry_wet_pct_min,dry_wet_pct_max_op,dry_wet_pct_max,dry_wet_pct_comments,steady_state,additional_comments,companion_tag,created_date,modified_date,old_terretox_result_number
0,61358,1108761,NaN,NC,NaN,NC,NaN,NC,NC,NaN,...,NC,NaN,NC,NaN,NC,,NaN,01/31/1985,02/21/1995,NaN
1,136246,1165989,NaN,NC,NaN,NC,NaN,NC,NC,NaN,...,NC,NaN,NC,NaN,NC,,NaN,04/19/1996,NaN,NaN
2,223703,1097736,NaN,NC,NaN,NC,NaN,NC,NC,NaN,...,NC,NaN,NC,NaN,NC,,NaN,10/02/2003,NaN,NaN
3,151106,1187635,NaN,NC,NaN,NC,NaN,NC,NC,NaN,...,NC,NaN,NC,NaN,NC,,NaN,10/10/1997,NaN,NaN
4,53559,1012894,NaN,NC,NaN,NC,NaN,NC,NC,NaN,...,NC,NaN,NC,NaN,NC,,NaN,11/21/1995,NaN,NaN


In [107]:
test = pd.read_csv('data/ecotox/tests.txt',sep='\|', engine='python')
test.head()

,test_id,reference_number,test_cas,test_grade,test_grade_comments,test_formulation,test_formulation_comments,test_radiolabel,test_radiolabel_comments,test_purity_mean_op,...,halflife_min_op,halflife_min,halflife_max_op,halflife_max,halflife_unit,halflife_comments,additional_comments,created_date,modified_date,published_date
0,1,35000,1336363,R,NaN,NR,NaN,NR,NaN,NaN,...,NaN,NC,NaN,NC,NC,NaN,,01/28/1998,07/20/2012,09/14/2012
1,2,35000,1336363,R,NaN,NR,NaN,NR,NaN,NaN,...,NaN,NC,NaN,NC,NC,NaN,,01/28/1998,07/20/2012,09/14/2012
2,3,35001,2385855,NR,NaN,NR,NaN,NR,NaN,NaN,...,NaN,NC,NaN,NC,NC,NaN,,01/28/1998,07/20/2012,09/14/2012
3,4,35002,7631994,NR,NaN,NR,NaN,NR,NaN,NaN,...,NaN,NC,NaN,NC,NC,NaN,,01/28/1998,07/20/2012,09/14/2012
4,5,35002,7631994,NR,NaN,NR,NaN,NR,NaN,NaN,...,NaN,NC,NaN,NC,NC,NaN,,01/28/1998,07/20/2012,09/14/2012


In [108]:
species = pd.read_csv('data/ecotox/validation/species.txt',sep='\|', engine='python')
species.head()

,species_number,common_name,latin_name,kingdom,phylum_division,subphylum_div,superclass,class,tax_order,family,genus,species,subspecies,variety,ecotox_group
0,1,Fathead Minnow,Pimephales promelas,Animalia,Chordata,Vertebrata,Osteichthyes,Actinopterygii,Cypriniformes,Cyprinidae,Pimephales,promelas,NaN,NaN,"Fish,Standard Test Species"
1,2,Bluegill,Lepomis macrochirus,Animalia,Chordata,Vertebrata,Osteichthyes,Actinopterygii,Perciformes,Centrarchidae,Lepomis,macrochirus,NaN,NaN,"Fish,Standard Test Species"
2,3,Brook Trout,Salvelinus fontinalis,Animalia,Chordata,Vertebrata,Osteichthyes,Actinopterygii,Salmoniformes,Salmonidae,Salvelinus,fontinalis,NaN,NaN,"Fish,Standard Test Species"
3,4,Rainbow Trout,Oncorhynchus mykiss,Animalia,Chordata,Vertebrata,Osteichthyes,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,mykiss,NaN,NaN,"Fish,Standard Test Species,U.S. Exotic/Nuisanc..."
4,5,Water Flea,Daphnia magna,Animalia,Arthropoda,Crustacea,NaN,Branchiopoda,Diplostraca,Daphniidae,Daphnia,magna,NaN,NaN,"Crustaceans,Standard Test Species"


**Prefilter**
Considering only:
- Endpoint LC50 or EC50 (Mortality at 50%)
- No embrions (Code EM)
- Group Fish

Considering all the endpoints containing LC50 or EC50: there are **different codes identify the same endpoints**

In [109]:
results[results.endpoint.str.contains("LC50")].endpoint.unique()

array(['LC50', 'LC50*', 'LC50/', 'LC50*/'], dtype=object)

In [110]:
results[results.endpoint.str.contains("EC50")].endpoint.unique()

array(['EC50', 'EC50/', 'EC50*', 'EC50*/'], dtype=object)

In [111]:
# Retaining only LC50 or EC50
res_50 = results[(results.endpoint.str.contains("LC50")) | (results.endpoint.str.contains("EC50"))]

Considering the possible effects connected to LC50. We need to filter on **Mortality Group**

In [112]:
res_50.effect.unique()

array(['ITX', 'MOR', 'BEH', 'GRO', 'REP', 'MLT', 'PHY', 'POP', 'BCM',
       'DVP', '~MOR', 'HIS', 'BEH/', 'REP/', 'GEN', '~ITX', 'ENZ', '~DVP',
       'MOR/', 'FDB', 'MPH', 'AVO', '~DVP/', 'INJ', 'HRM', 'CEL', '~REP/',
       'ITX/', 'PRS', 'GRO/', 'POP/', 'PHY/', '~FDB', '~MOR/', '~PHY',
       '~GRO', '~REP', 'IMM', '~BCM', 'NER', 'ENZ/', 'BCM/', 'ACC',
       'FDB/', '~GEN', '~POP', '~MLT', 'DVP/', '~ITX/', '~ACC', '~BEH'],
      dtype=object)

*Note*: ~MOR is **retarted mortality**. MOR/ should be the same as MOR (?)

In [113]:
# Only the ones inside mortality group
res_50_mor = res_50[res_50.effect.str.contains("MOR")]

Look at the possible measurement inside the Mortality Group. **We take them all for now: we may filter only for mortality measurement?**

In [114]:
res_50_mor.measurement.unique()

array(['MORT', 'MORT/', 'SURV', 'TKNO', 'HTCH', 'SURV/', 'SVVS', 'LBCN',
       'HTCH/', 'LFSP', 'GMOR/', 'GMOR', 'LIFE', 'EBCN', 'MDTH', 'LIFE/'],
      dtype=object)

Retaining only **Fish not embrion**

In [115]:
# No embrions in the test
test_no_EM = test[test.organism_lifestage != "EM"]

In [116]:
# No Fish
species = species[~species.ecotox_group.isnull()]
species_fish = species[species.ecotox_group.str.contains("Fish")]

# Merge
test_fish_only = test.merge(species_fish, on="species_number")

Combining **all together**

In [117]:
results_final = res_50_mor.merge(test_fish_only, on = "test_id")

In [118]:
results_final.index.size

64746

## Looking for null values, outliers and strange units in the new datasets

### CAS : Foreign key to CHEMICALS

In [100]:
# It's an index -> shouldn't be null values
sum(results_final['test_cas'].isnull())

0

No sense talking about outliers (possible with ECOTOX_GROUP when we will talk about the CAS characteristics)

**Defining a function to count the NaN and the possible outputs for one of the interested big features**

In [101]:
def null_output_counts(feature, dataframe):
    
    # Find columns that start with the interesting feature
    cols = list(dataframe.columns)
    features_interested = [i for i in cols if i.startswith(feature)]
    df_nan  = pd.DataFrame (index = features_interested, columns = ['null_values_inc_NC_NR%', '#outputs'])
    
    #Count total NaN + NR + NC
    for i in features_interested:
        df_nan['null_values_inc_NC_NR%'][i] = (sum(dataframe[i].isnull()) + len(dataframe[dataframe[i] == "NR"]) + len(dataframe[dataframe[i] =="NC"]))/len(dataframe)*100
        df_nan['#outputs'][i] = len(dataframe[i].unique())
    return df_nan, features_interested

### ORGANISMS

In [102]:
# Apply defined function to organism
df_org_nan, org_list = null_output_counts("organism", results_final)
df_org_nan

,null_values_inc_NC_NR%,#outputs
organism_final_wt_mean_op,100,1
organism_final_wt_mean,99.9707,6
organism_final_wt_min_op,100,1
organism_final_wt_min,99.9722,8
organism_final_wt_max_op,100,1
organism_final_wt_max,99.9722,8
organism_final_wt_unit,99.966,4
organism_final_wt_comments,100,1
organism_habitat,0,1
organism_source,89.7986,9


**Usable features**: *organism_lifestage*, *organism_age_mean* (?)

Good to see at the correlation, need usable **numeric** features.  
Considering the organism_lifestage and age_mean (with age unit feature needed)

In [103]:
#consider only interesting features
org_intrs = ["organism_lifestage", "organism_age_mean", "organism_age_unit"]
df_org = results_final[org_intrs]

In [104]:
df_org.organism_lifestage.value_counts(dropna=False).head(20)

NR    38329
JV     6412
FI     3944
LV     2899
NC     2827
FY     2584
AD     2003
EG     1093
EM      959
SU      542
SF      501
YY      476
YO      321
AL      265
MX      257
YE      196
EY      154
LR       99
PA       96
UY       92
Name: organism_lifestage, dtype: int64

A lot of NR and NC, but once dropped/imputed (I don't see how imputing a categorical like this one) seems a useful feature

In [194]:
organism_unit_counts = df_org.organism_age_unit.value_counts(dropna=False)
organism_unit_counts

NR      97911
h       13099
d       12504
ins      9382
NC       4250
mo       3498
wk       3375
stg      1214
dph      1175
yr        374
go        326
yc        272
hph       252
ls        234
dpu       193
hpe       178
ge        170
hpf       153
dpe       147
fd        117
dpf       101
gs         88
nf         72
LSI        65
mpf        52
wpe        46
ac         40
mpp        24
cs         21
wph        18
dps        16
tls        15
hbf         9
sst         7
dpw         7
mph         6
NaN         5
hpr         5
dpp         5
gts         5
brd         4
su          3
mi          2
dpmm        2
dpo         2
moph        1
mopm        1
dpys        1
dpr         1
Name: organism_age_unit, dtype: int64

Also now, lot of strage unit variables.  
Considering NR and NC to see if there's a match with missing means:

In [200]:
df_org[df_org.organism_age_unit == "NC"].organism_age_mean.value_counts(dropna=False)

NC    4250
Name: organism_age_mean, dtype: int64

In [201]:
df_org[df_org.organism_age_unit == "NR"].organism_age_mean.value_counts(dropna=False)

NR    97910
3         1
Name: organism_age_mean, dtype: int64

NC totally match, NR not (But without unit...3 what?)

Some strange unit values used here:
- ins: instar
- stg: stage 
- dph: days post hatch
- go: gosner stage
- yc: year class
- hph: hours post hatch

Full list in **data/DataCodes.pdf (Appendix I)**

Looking to the top cases:

In [204]:
df_org[df_org.organism_age_unit == "ins"].organism_age_mean.value_counts(dropna=False)

4          4120
3          1690
1          1249
NR         1210
2           953
5            80
6            33
NaN           9
12            8
Z3            7
4TH           6
3RD           3
13            2
7             2
LAST          2
Z1            2
8             1
3RD-4TH       1
10            1
11            1
9             1
LATE          1
Name: organism_age_mean, dtype: int64

In [205]:
df_org[df_org.organism_age_unit == "stg"].organism_age_mean.value_counts(dropna=False)

4         331
1         229
2         150
NR        129
3          99
25         51
8          32
6          24
5          23
7          12
10         12
26         11
46         11
C          10
9           7
C OR D      6
6A          6
21          6
CI-CIV      4
11          4
53          4
Z1          4
49          4
47          4
45          4
48.5        4
15          4
20          3
12          3
C1-C3       2
39          2
28          2
13          2
42          2
14          2
D3-D4       1
16          1
C4          1
30          1
35          1
D1-D2       1
L4          1
A-B         1
24          1
D0          1
54          1
Name: organism_age_mean, dtype: int64

In [206]:
df_org[df_org.organism_age_unit == "dph"].organism_age_mean.value_counts(dropna=False)

NR     465
14     254
4       50
1       38
7       38
      ... 
167      1
46       1
43       1
41       1
74       1
Name: organism_age_mean, Length: 63, dtype: int64

In [207]:
df_org[df_org.organism_age_unit == "go"].organism_age_mean.value_counts(dropna=False)

25    174
NR    110
26     20
36      8
19      5
46      4
11      2
21      1
29      1
33      1
Name: organism_age_mean, dtype: int64

In [208]:
df_org[df_org.organism_age_unit == "yc"].organism_age_mean.value_counts(dropna=False)

2    152
1    120
Name: organism_age_mean, dtype: int64

In [210]:
df_org[df_org.organism_age_unit == "hph"].organism_age_mean.value_counts(dropna=False)

24    181
8      17
12     16
NR     10
1       9
72      6
4       4
7       4
48      2
2       2
3       1
Name: organism_age_mean, dtype: int64

In [211]:
df_org[df_org.organism_age_unit == "ls"].organism_age_mean.value_counts(dropna=False)

NR    138
2      38
4      30
1      16
3      11
6       1
Name: organism_age_mean, dtype: int64

**Here we don't have a lot of NaN**:
in my opinion if we want to use this feature we need to:
1. Put all in the same unit of measure or
2. Categorize the most important units

### EXPOSURES

In [213]:
# Computing dataframe for exposure
df_exp_nan, exp_list = null_output_counts("exposure", test)
df_exp_nan

,null_values_inc_NC_NR%,#outputs
exposure_duration_mean_op,99.6701,6
exposure_duration_mean,73.6149,176
exposure_duration_min_op,99.9973,2
exposure_duration_min,99.4814,31
exposure_duration_max_op,99.9973,2
exposure_duration_max,99.4814,38
exposure_duration_unit,72.7731,29
exposure_duration_comments,99.8247,18
exposure_type,8.4424,62
exposure_type_comments,99.8802,8


**73%** of the exposure_duration_mean (most interesting feature here are **missing**).  
Moving on considering duration_unit and mean 

In [164]:
exp_intrs = ["exposure_duration_unit", "exposure_duration_mean"]
df_exp = test[exp_intrs]

Counting the number of each value:

In [161]:
exposure_duration_unit_counts = df_exp.exposure_duration_unit.value_counts(dropna=False)
exposure_duration_unit_counts

NC      108744
h        27017
d        11308
wk        1275
mi         385
em         260
ht          93
dpe         57
cd          56
s           45
lva         41
mo          36
lvp         22
wpe         21
NaN         14
dph         11
yr           9
dpgm         8
ma           8
ins          7
ge           6
hv           6
ep           6
brd          3
gm           3
hpf          2
lf           2
hpe          2
stg          1
Name: exposure_duration_unit, dtype: int64

**108744 NC + 14 NaN**.  
Reflecting in exposure_duration_mean:

In [167]:
df_exp[df_exp.exposure_duration_unit == "NC"].exposure_duration_mean.value_counts(dropna=False)

NC    108744
Name: exposure_duration_mean, dtype: int64

Same number: **all NC** from one side have the same NC in duration_mean

Duration unit meanings:
- h/d/wk/mi/s/mo/yr = hour/day/weeks/minute/seconds/month/years
- em: to emergence
- ht: until hatch
- dpe: days post-emergence
- cd: colony diameter
- lva: larva to adult
- lvp: larva to pupa
- wpe: weeks post emergence
- dph: days post hatch
- dpgm: days post germination
- ma: to maturity
- ins: instar
- ge: generation
- hv: harvest
- ep: egg to pupation
- brd: brood or litter
- hpf: hours post fertilization
- lf: lifetime (no associated numeric value)
- hpe: hours post emergence
- stg: stage  

Full list in **data/DataCodes.pdf (Appendix I)**

Considering the top cases for strange units:

In [168]:
df_exp[df_exp.exposure_duration_unit == "em"].exposure_duration_mean.value_counts(dropna=False)

NaN    246
NR      14
Name: exposure_duration_mean, dtype: int64

In [169]:
df_exp[df_exp.exposure_duration_unit == "ht"].exposure_duration_mean.value_counts(dropna=False)

NaN    85
NR      8
Name: exposure_duration_mean, dtype: int64

In [170]:
df_exp[df_exp.exposure_duration_unit == "dpe"].exposure_duration_mean.value_counts(dropna=False)

14    48
2      4
3      4
5      1
Name: exposure_duration_mean, dtype: int64

In [171]:
df_exp[df_exp.exposure_duration_unit == "cd"].exposure_duration_mean.value_counts(dropna=False)

NaN    56
Name: exposure_duration_mean, dtype: int64

In [173]:
df_exp[df_exp.exposure_duration_unit == "lva"].exposure_duration_mean.value_counts(dropna=False)

NaN    41
Name: exposure_duration_mean, dtype: int64

In [174]:
df_exp[df_exp.exposure_duration_unit == "lvp"].exposure_duration_mean.value_counts(dropna=False)

NaN    22
Name: exposure_duration_mean, dtype: int64

In [175]:
df_exp[df_exp.exposure_duration_unit == "dph"].exposure_duration_mean.value_counts(dropna=False)

28    8
4     2
1     1
Name: exposure_duration_mean, dtype: int64

In [176]:
df_exp[df_exp.exposure_duration_unit == "dpgm"].exposure_duration_mean.value_counts(dropna=False)

21    8
Name: exposure_duration_mean, dtype: int64

In [177]:
df_exp[df_exp.exposure_duration_unit == "ma"].exposure_duration_mean.value_counts(dropna=False)

NaN    8
Name: exposure_duration_mean, dtype: int64

**Many of this cases have just NaN or NR** -> What should we do with this feature?

### CONCENTRATIONS

In [216]:
# Computing dataframe for concentrations
df_conc_nan, conc_list = null_output_counts("conc", results)
df_conc_nan

,null_values_inc_NC_NR%,#outputs
conc1_type,3.81489,8
conc1_mean_op,89.7413,6
conc1_mean,4.37073,22896
conc1_min_op,96.6921,5
conc1_min,49.3594,15014
conc1_max_op,96.6824,5
conc1_max,49.3588,17726
conc1_unit,0.0307798,239
conc1_comments,93.6974,598
conc2_type,98.8509,6


Only **conc1 seems useful**:

In [217]:
# Computing dataframe for concentration 1
df_conc_nan, conc_list = null_output_counts("conc1", results)
df_conc_nan

,null_values_inc_NC_NR%,#outputs
conc1_type,3.81489,8
conc1_mean_op,89.7413,6
conc1_mean,4.37073,22896
conc1_min_op,96.6921,5
conc1_min,49.3594,15014
conc1_max_op,96.6824,5
conc1_max,49.3588,17726
conc1_unit,0.0307798,239
conc1_comments,93.6974,598


As usual, considering the **mean and unit**. Useful to look also to type:

In [218]:
conc_intrs = ['conc1_type', 'conc1_mean', 'conc1_unit']
df_conc = results[conc_intrs]

In [220]:
df_conc.conc1_type.value_counts()

A     70512
F     43190
T     41105
NC     6307
D      3294
U       972
L       299
NR       14
Name: conc1_type, dtype: int64

**Small number of NaN, few categories: could be useful as categorical**
Definitions:
- A: Active ingredients
- F: Formulation
- D: Dissolved
- L: Labile
- T: Total
- U: Unionized

Considering now units:

In [224]:
conc_unit_counts = df_conc.conc1_unit.value_counts(dropna=False)
conc_unit_counts

ug/L             78499
mg/L             30896
ppm              14452
AI ug/L           6174
AI mg/L           5170
                 ...  
mg/L/h               1
mg/L 10 mi           1
ng/g egg             1
nmol/kg              1
AI ug/100 cm2        1
Name: conc1_unit, Length: 239, dtype: int64

**239 different concentrations**: how to deal with all these different units?  
**_AI_** is active ingredients -> What's the difference with the normal case?

In [227]:
conc_unit_counts.head(30)

ug/L              78499
mg/L              30896
ppm               14452
AI ug/L            6174
AI mg/L            5170
ppb                4052
uM                 2356
ug/ml              1716
mg/kg              1461
ul/L               1355
mM                 1069
umol/L             1056
AI ppm food        1055
AI ppm              916
mg/kg soil          827
mg/kg dry soil      824
%                   695
nM                  628
AI ug/ml            573
ng/L                507
ug/cm2              490
ug/g soil           446
AI %                426
mmol/L              422
g/L                 388
mg/ml               386
ae mg/L             380
M                   378
nmol/L              363
ug/eu               358
Name: conc1_unit, dtype: int64

As we see, there are a lot of different unit and all makes sense. More details in **data/DataCodes.pdf (Appendix N & W)**

A closer look to the first case:

In [225]:
df_conc[df_conc.conc1_unit == "ug/L"].conc1_mean.value_counts(dropna=False)

NR         3885
1000        758
10000       683
100000      544
100         485
           ... 
5.40          1
7460000       1
552.8         1
305900        1
55190         1
Name: conc1_mean, Length: 12428, dtype: int64

**Concentration is a variable we need for sure**. We should fix it, proposed solutions are the same as for organisms:
- Considering a categorical definition
- All in the same units